# Meta Analysis Notebook

## Analysis Utilities
The following functions are built to aid the development of helpful visualizations. These functions load the metadata.json (meta-data) file associated with the data analysis. 

- *getData* will load the metadata JSON file. 
- *getCellData* will load the different cell versions into a dataframe. Each cell version is labeled with a commit_id and time stamp. 
- *getNotebookData* will load the metrics generated by Cynosure for each notebook version. Again, each notebook version is labeled with a commit_id and a time stamp. 
- *convertToDate* and convertUnixTSToDate handle the conversion of different time-stamps into human-readable Python DateTime. 
- *createDataFrame* creates the data frames associated with cells versions and notebook versions. This function also calculates the number of items in columns that contain lists of items like 'vars' and 'tags'.

In [ ]:
from enum import unique
import json
import pandas as pd
from datetime import datetime
from dateutil import parser
import numpy as np
import altair as alt
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

def getData():
    with open('meta-data.json') as fp:
        data = json.load(fp)
    
    return data
  
def getCellData(data): 
    vers = []
    versions = data['commits'].keys()
    for version in versions:
        notebooks = data["commits"][version]['notebooks']
        for notebook in notebooks:
            cells = notebook['contents']['cells']
            for i, cell in enumerate(cells):
                info = dict(cell)

                info.pop('metadata')
                info.update(cell['metadata'])
                
                info['cell_index'] = chr(i + 97)
                info['notebook'] = notebook['name']
                info['commit_id'] = version
                
                info['time'] = data['commits'][version]['time']
                vers.append(info)
    
    return vers

def getNotebookData(data):
    analysis = []
    versions = data['commits'].keys()
    for version in versions:
        notebooks = data['commits'][version]['notebooks']
        for notebook in notebooks:
            info = notebook['analysis']
            info['commit_id'] = version
            info['notebook'] = notebook['name']
            info['time'] = data['commits'][version]['time']
            analysis.append(info)
        
    return analysis

def convertToDate(isostr):
    if (str(isostr) != 'nan'):
        return parser.parse(isostr)

def convertUnixTSToDate(unixstr):
    return datetime.fromtimestamp(unixstr)

def createDataFrame():

    notebook_data = getNotebookData(data)
    dfNb = pd.DataFrame.from_dict(notebook_data)
    
    cell_data = getCellData(data)
    dfCell = pd.DataFrame.from_dict(cell_data)
    dfCell['created-datetime'] = dfCell['created-datetime'].apply(convertToDate)
    dfCell['time'] = dfCell['time'].apply(convertUnixTSToDate)
    dfNb['time'] = dfNb['time'].apply(convertUnixTSToDate)
    dfCell['deps_len'] = np.where(dfCell['deps'].str.len() > 0, dfCell['deps'].str.len(), 0)
    dfCell['tags_len'] = np.where(dfCell['tags'].str.len() > 0, dfCell['tags'].str.len(), 0)
    dfCell['vars_len'] = np.where(dfCell['vars'].str.len() > 0, dfCell['vars'].str.len(), 0)
    dfCell['api_len'] = np.where(dfCell['api_calls'].str.len() > 0, dfCell['api_calls'].str.len(), 0)
    dfCell['func_len'] = np.where(dfCell['func_definitions'].str.len() > 0, dfCell['func_definitions'].str.len(), 0)
    dfCell['output_len'] = np.where(dfCell['outputs'].str.len() > 0, dfCell['outputs'].str.len(), 0)

    # dfCell.to_csv('cell_data.csv')
    # dfNb.to_csv('notebook_data.csv')
    
    return (dfCell, dfNb)

data = getData()

### dfCell

The dfCell dataframe identifies every version of every cell in the analysis notebook. 
Each row in this dataframe corresponds to a single version of a single cell found in the analysis notebook. 

The dfCell dataframe has 31 attributes. They are:

| Attribute Number | Attribute Name | Definition |
| --- | --- | --- |
| 0 | execution_count | This metric is generated by the Jupyter platform to designate both the number of times a cell has been executed and the order in which it appears next to other executed cells. |
| 1 | id | This is a GUID generated by Cynosure to identify unique cells irrespective of the type of cell created. |
| 2 | cell_type | Identifies the type of cell created. Can be "markdown", "code", "raw" |
| 3 | source | The contents of the cell. Each line (ending with a new line character) is an item in this list. |
| 4 | source_length | The number of "lines" found in the cell contents. |
| 5 | outputs | The output contents as stored by the Jupyter platform. |
| 6 | hash | A hash of the cell contents to enable comparison between cells. |
| 7 | comments | The list of individual comments found in each cell. |
| 8 | ln_comment_count | The number of single line comments made using the '#' character. |
| 9 | multiln_comment_count | The number of multi-line comments made using the ''' characters. |
| 10 | vars | A list of code variables defined in the cell found using the Python AST library. |
| 11 | api_calls | A list of api calls found in the cell using regular expressions. |
| 12 | func_definitions | A list of python function definitions found in the cell using Python AST. |
| 13 | created-datetime | The date-time when the cell was created. |
| 14 | deps | The other cell on which this cell is dependent upon. |
| 15 | executions | The number of times the cell was cumulatively executed by the author. |
| 16 | keystrokes | The number of times the cell was cumulatively edited by the author. |
| 17 | modified-datetime | The date-time when the cell was last edited. |
| 18 | tags | The tags associated with the cell. |
| 19 | cell_index | The unique alpha numeric character associated with the cell. |
| 20 | notebook | The name of the notebook in which the cell was found. |
| 21 | commit_id | The unique id associated with a version of the cell. |
| 22 | time | The date-time when the cell version was saved. |
| 23 | metadata | The original dictionary structure of the meta-data associated with the cell. |
| 24 | collapsed | A boolean showing whether the author collapsed the cell from being seen. |
| 25 | jupyter | The version of jupyter used to edit this cell. |
| 26 | deps_len | The number of dependencies of this cell. |
| 27 | tags_len | The number of tags associated with this cell. |
| 28 | vars_len | The number of variables in this cell. |
| 29 | api_len | The number of api calls found in this cell. |
| 30 | func_len | The number of function definitions found in this cell. |
| 31 | output_len | The number of outputs from this cell. |

### dfNb

The dfNb dataframe identifies every version of the analysis notebook. 
Each row in this dataframe corresponds to a single version of the notebook found in the project directory. 

| Attribute Number | Attribute Name | Definition |
| --- | --- | --- |
| 0 | lang | The language the notebook was implemented in. |
| 1 | avg_markdown_lang | The natural language (e.g. english, spanish, japanese) detected across cells in the notebook. |
| 2 | ds_api_calls | A list of all API calls associated with common Data Science libraries. |
| 3 | api_calls | A list of all API calls found in the notebook. |
| 4 | libraries | A list of all data science Libraries (imports) found in the notebook. |
| 5 | ast_connection | A score prescribed by Python AST of the relationship between code cells. | 
| 6 | all_libs | A list of all libraries (imports) found in the notebook.
| 7 | date | The day when the notebook version was created. |
| 8 | hashes | A list of hashes of the notebook cells to make it easy to compare cells. |
| 9 | exec_order | A list of execution counts of notebook cells. |
| 10 | exec_gap | The gap in the execution counts in notebook cell, used to indicate a break in the linearity of execution. |
| 11 | no_exec_cells | Number of cells with no execution counts. |
| 12 | exec_max | The maximum execution count. |
| 13 | exec_min | The minimum execution count. |
| 14 | code_count | The ratio of lines found in code cells (with output) to total lines in the notebook. |
| 15 | text_count | The ratio of lines found in markdown cells to total lines in the notebook. |
| 16 | raw_count | The ratio of lines found in raw cells to total lines in the notebook. |
| 17 | noout_count | The ratio of lines found in code cells (without output) to total lines in the notebook. |
| 18 | text_headers | The number of text headers. |
| 19 | code_lines | The number of lines of code found in code cells. |
| 20 | text_lines | The number of lines of markdown found in markdown cells. |
| 21 | raw_lines | The number of lines found in raw cells. |
| 22 | noout_lines | The number of lines found in code cells with no output. |
| 23 | c_tot_lines | The number of lines found in the notebook excluding cells without execution history. |
| 24 | tot_lines | Total number of lines of code or text across all cells. |
| 25 | comment_count | The total number of independent comments within code cells. |
| 26 | comment_lines | The total number of lines of comments within code cells. |
| 27 | media_count | The total number of output. |
| 28 | media_text | The total number of text output. |
| 29 | media_image | The total number of image output. |
| 30 | media_table | The total number of table output. |
| 31 | media_error | The total number of output errors. |
| 32 | avg_media_text | The ratio of code output that displayed text to all output. |
| 33 | avg_media_image | The ratio of code output that displayed images to all output. |
| 34 | avg_media_table | The ratio of code output that displayed a table to all output. |
| 35 | avg_media_error | The ratio of code output that resulted in an error to all output. |
| 36 | c_code_count | The ratio of code cell with output to total cells. |
| 37 | c_text_count | The ratio of markdown cells with no output to total cells. |
| 38 | c_noout_count | The ratio of code cells with no output to total cells. |
| 39 | c_raw_count | The ratio of raw cells to total cells. |
| 40 | tot_count | The total number of cells, excluding the cells with no execution history. |
| 41 | code_cells | The total number of code cells that did have an output. |
| 42 | text_cells | The total number of markdown cells. |
| 43 | noout_cells | The total number of code cells that did not have an output. |
| 44 | raw_cells | The total number of raw cells. |
| 45 | tot_cells | The total number of cells. |
| 46 | code_space | The number of space characters in code cells that did have an output. |
| 47 | text_space | The number of space characters in markdown cells in the notebook. |
| 48 | raw_space | The number of space characters in raw cells in the notebook. |
| 49 | noout_space | The number of space characters in code cells that did not have an output. |
| 50 | file_size | The file size of the notebook. |
| 51 | commit_id | Version ID of the notebook. |
| 52 | notebook | Name of the notebook. |
| 53 | time | Time when the version was created. |

In [ ]:
dfCell, dfNb = createDataFrame()

## Cell Analysis

This section focuses on analyzing and visualizing changes in cells within the notebook. 

In [ ]:
# Isolate cells with code
tmp = dfCell[dfCell['cell_type'] == 'code'].dropna(subset=['executions', 'keystrokes'])
tmp.set_index(['id', 'time', 'cell_index', 'commit_id'], inplace=True)
tmp.sort_index(inplace=True)

tmp['diffs_exec'] = np.nan
tmp['diffs_ks'] = np.nan
tmp['diffs_apis'] = np.nan

# Determine the time frames/ commits when the number of executions
# and the number of key strokes changed (when the author made and executed changes)

# This process generates a warning because of the way in which we are slicing 
# the dataframe. (Ignore)
dfCellCode = pd.DataFrame()
for idx in tmp.index.levels[0]:
    tmp.diffs_exec[idx] = tmp.executions[idx].diff()
    tmp.diffs_ks[idx] = tmp.keystrokes[idx].diff()
    tmp.diffs_apis[idx] = tmp.api_len[idx].diff()
    
# populate a new dataframe with this information
dfCellCode['executions'] = tmp['diffs_exec']
dfCellCode['keystrokes'] = tmp['diffs_ks']
dfCellCode['apis'] = tmp['diffs_apis']

dfCellCode['dependencies'] = tmp['deps_len']
dfCellCode['tags'] = tmp['tags_len']
dfCellCode['variables'] = tmp['vars_len']
dfCellCode['functions'] = tmp['func_len']
dfCellCode['outputs'] = tmp['output_len']

# drop rows where executions and keystrokes were not defined
dfCellCode = dfCellCode.dropna(subset=['executions', 'keystrokes'])

# create another dataframe where we only look at non-zero execution or keystrokes
dfCellCode_active = dfCellCode[(dfCellCode['executions'] > 0) | (dfCellCode['keystrokes'] > 0)]

# save to file (optional)
# dfCellCode.to_csv('code_cell_diff_data.csv')
# dfCellCode_active.to_csv('code_cell_diff_data_active.csv')

### Determine time-frame when author made the most changes to code in terms of executing them, and editing them. 

In [ ]:
alt.Chart(dfCellCode_active.reset_index(), title='Time frame when author executed code cells.').mark_point().encode(
    x=alt.X('time:T', axis=alt.Axis(labelAngle=0)),
    y='executions:Q',
    color='cell_index:N'
)

In [ ]:
alt.Chart(dfCellCode_active.reset_index(), title='Time frame when author made changes to code cells.').mark_point().encode(
    x=alt.X('time:T', axis=alt.Axis(labelAngle=0)),
    y='keystrokes:Q',
    color='cell_index:N'
)

In [ ]:
alt.Chart(dfCellCode_active.reset_index(), title='Time frame when author added API calls to code cells.').mark_point().encode(
    x=alt.X('time:T', axis=alt.Axis(labelAngle=0)),
    y='apis:Q',
    color='cell_index:N'
)

#Note: Negative changes in the APIs indicate that the author was removing API calls from the cells.

In [ ]:
alt.Chart(dfCellCode_active.reset_index(), title='Time frame when author executed code cells after making changes to them.').mark_circle().encode(
    x=alt.X('time:T', axis=alt.Axis(labelAngle=0)),
    y='executions:Q',
    size='keystrokes:Q',
    color='cell_index:N'
)

### Determine the code cells which the authors grouped together in execution and in editing.

In [ ]:
alt.Chart(dfCellCode.reset_index(), title='Cells that are executed together within each time frame.').mark_rect().encode(
    x='time:O',
    y='cell_index:O',
    color='executions:Q'
)

In [ ]:
alt.Chart(dfCellCode.reset_index(), title='Cells that are edited together within each time frame.').mark_rect().encode(
    x='time:O',
    y='cell_index:O',
    color='keystrokes:Q'
)

In [ ]:
alt.Chart(dfCellCode.reset_index(), title='Changes in the number of API calls in code cells within each time frame.').mark_rect().encode(
    x='time:O',
    y='cell_index:O',
    color='apis:Q'
)

### Determine the life-span of cells.

In [ ]:
# Create a simpler dataframe with start and end times of the cell's life-span
# Find all unique cell IDs
unique_cell_id = dfCell.id.unique()
cell_duration = []

# Iterate through each cell
for id in unique_cell_id:
    duration = {}
    duration['id'] = id
    cell = dfCell[dfCell['id'] == id]
    cell_index = np.array(cell['cell_index'].unique())
    times = np.array(cell['time'])
    
    # assign the start and end duration of cell
    # cells will recieve the last time-stamp in 
    # which they continue to exist.
    duration['type'] = np.array(cell['cell_type'].unique())[0]
    duration['start'] = times[0]
    duration['end'] = times[-1]
    duration['cell_index'] = cell_index[0]
    cell_duration.append(duration)

cd = pd.DataFrame(cell_duration)
cd = cd.sort_values(by = 'cell_index')
cd

In [ ]:
alt.Chart(cd, title='Life span of each cell. Colors indicate code type.').mark_bar().encode(
    x='start',
    x2='end',
    y='cell_index',
    color='type'
)

In [ ]:
# Create a simpler dataframe with start and end times of the cell's life-span
# Find all unique cell IDs
unique_cell_id = dfCell.id.unique()
cell_duration = []

# Iterate through each cell
for id in unique_cell_id:
    duration = {}
    duration['id'] = id
    cell = dfCell[dfCell['id'] == id]
    cell_index = np.array(cell['cell_index'].unique())
    times = np.array(cell['time'])
    
    # we will define life span of a cell based on when it was created
    # and the last time it was modified.
    createdtime = np.array(cell['created-datetime'])
    modifiedtime = np.array(cell['modified-datetime'])
    
    # assign the start and end duration of cell
    # cells will recieve the last time-stamp in 
    # which they continue to exist.
    duration['type'] = np.array(cell['cell_type'].unique())[0]
    duration['start'] = createdtime[0]
    duration['end'] = modifiedtime[-1]
    duration['cell_index'] = cell_index[0]
    cell_duration.append(duration)

cd = pd.DataFrame(cell_duration)
cd = cd.sort_values(by = 'cell_index')

In [ ]:
alt.Chart(cd, title='Life span of each cell based on when they are created and last-modified.').mark_bar().encode(
    x='start',
    x2='end',
    y='cell_index',
    color='type'
)

### Determine the life-span variables and APIs defined by the author. 

In [ ]:
flat_vars = []
flat_apis = []

# For each cell 
for id in unique_cell_id:
    cell = dfCell[dfCell['id'] == id]
    cell_index = (np.array(cell['cell_index'].unique()))[0]
    var = np.array(cell['vars'])
    api_calls = np.array(cell['api_calls'])
    cids = np.array(cell['commit_id'])
    times = np.array(cell['modified-datetime'])

    # For each variable found in each cell (skips if cell is markdown) 
    for i, x in enumerate(var):
        if (type(x) == list):
            for vx in x:
                v = {}
                v['id'] = id
                v['cell_index'] = cell_index
                v['var'] = vx 
                v['commit_id'] = cids[i]
                v['time'] = times[i]
                flat_vars.append(v)

    # For each api call found in each cell (skips if cell is markdown)
    for j, x in enumerate(api_calls):
        if (type(x) == list):
            for ax in x:
                a = {}
                a['id'] = id
                a['cell_index'] = cell_index
                a['api'] = ax
                a['commit_id'] = cids[j]
                a['time'] = times[j]
                flat_apis.append(a)


flv = pd.DataFrame(flat_vars)

vars = np.array(flv['var'].unique())
var_life_span = []

# For each variable
# construct a life span dataframe
for v in vars:
    ls = {}
    ls['var'] = v

    for_var = flv[flv['var'] == v]
    times = np.array(for_var['time'])
    cell_index = np.array(for_var['cell_index'])[0]

    ls['cell_index'] = cell_index
    ls['start_time'] = times[0]
    ls['end_time'] = times[-1]
    var_life_span.append(ls)

fla = pd.DataFrame(flat_apis)

apis = np.array(fla['api'].unique())
api_life_span = []

# For each APIs
# construct a life-span dataframe
for v in apis:
    ls = {}
    ls['api'] = v

    for_api = fla[fla['api'] == v]
    times = np.array(for_api['time'])
    cell_index = np.array(for_api['cell_index'])[0]

    ls['cell_index'] = cell_index
    ls['start_time'] = times[0]
    ls['end_time'] = times[-1]
    api_life_span.append(ls)

var_data_life_span = pd.DataFrame(var_life_span)
api_data_life_span = pd.DataFrame(api_life_span)

var_data_life_span

In [ ]:
api_data_life_span = api_data_life_span.sort_values(by='cell_index')
alt.Chart(api_data_life_span, title='A rough measure of when the APIs were modified based on when respective cells were last modified time.').mark_bar().encode(
    x='start_time:T',
    x2='end_time:T',
    y='api:N',
    color='cell_index:N'
)

In [ ]:
var_data_life_span = var_data_life_span.sort_values(by='cell_index')
alt.Chart(var_data_life_span, title='A rough measure of when the variables were modified based on when respective cells were last modified time.').mark_bar().encode(
    x='start_time:T',
    x2='end_time:T',
    y='var:N',
    color='cell_index:N'
)

## Notebook Analysis

In [ ]:
alt.Chart(dfNb, title='Time frame when author experienced the most errors.').mark_point().encode(
    x='time:T',
    y='media_count:Q',
    size='media_text:Q'
)

## Cell Dependency Analysis

With Cynosure, authors can track the dependencies between cells. This is particularly important to track since notebooks do not provide a guide on the order of execution for cells. Visualizing dependencies can be an easy way to track authors' dead-end analytical steps/ techniques. 

In [ ]:
from pyvis import network as net
import networkx as nx
from ipywidgets import interact

# Get the contents of the cell for a particular version
def getSource(notebook, commit, cell_id):
    
    dfCommit = dfCell[dfCell['commit_id'] == commit]
    cell = dfCommit[dfCommit['id'] == cell_id]
    if (len(cell) > 0):
        cell_index = (cell['cell_index']).iat[0]
        source = ''.join(cell['source'].iat[0])
        
        return (source, cell_index)
    
    # error handling
    return ('', cell_id)

# show dependency graph of cells in the notebook
commits = list(data['commits'].keys())
nbs = list(data['notebook_cell_graphs'].keys())
print('Use slider to select commit and notebook')
@interact(index=(0, len(commits) - 1), nbIndex=(0, len(nbs) - 1))
def showDependencyData(index, nbIndex):
    commit = commits[index]
    notebook = nbs[nbIndex]
    print('commit id: ' + commit)
    print('commit time: ' + convertUnixTSToDate(data['commits'][commit]['time']).strftime("%Y-%m-%d %H:%M"))
    print('notebook: ' + notebook)
    
    g = net.Network(notebook=True, height='800px', width='100%')
    
    # construct one node for every cell
    edges = []
    nodes = data['notebook_cell_graphs'][notebook]['nodes'].keys()
    
    node_rename = {}
    node_color = []
    node_names = []
    node_titles = []
    
    # markdown cells are green, code cells are black
    metadata_node_color = '#00ff1e'
    code_node_color = '#162347'
    for i, n in enumerate(nodes):
        node = data['notebook_cell_graphs'][notebook]['nodes'][n]
        
        if (commit in node['commits']):
            source, cell_index = getSource(notebook, commit, node['id'])
            
            if (str(cell_index) != 'None'):
                node_rename[n] = cell_index
                node_titles.append(source)
                node_names.append(cell_index)

                if (node['type'] == 'code'):
                    node_color.append(code_node_color)
                else:
                    node_color.append(metadata_node_color)        
    
    # construct one edge for every cell to cell dependency
    edges_raw = data['notebook_cell_graphs'][notebook]['edges'].keys()
    node_ognames = list(node_rename.keys())
    for e in edges_raw:
        edge = data['notebook_cell_graphs'][notebook]['edges'][e]
        source = edge['source']
        target = edge['target']
        if (commit in edge['commits'] and (source in node_ognames and target in node_ognames)):
            edges.append((node_rename[source], node_rename[target]))

    # add nodes and edges to pyvis
    g.add_nodes(node_names, color=node_color, title=node_titles)
    for e in edges:
        g.add_edge(e[0], e[1], value=1)

    return g.show('example.html')